# Cyanobiome phylogenomics

In [3]:
import glob
import os
import pandas as pd
from Bio import SeqIO
from natsort import natsorted
import subprocess
import numpy as np
import re
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
from Bio import Phylo
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
!mkdir ./pnas-phylogenomics-top_100/

## Renaming genomes

In [6]:
def split_string_at(s, c, n):
    words = s.split(c)
    return c.join(words[:n]), c.join(words[n:])

def extract_ncbi_taxa(inputs_folder):
    input_list = glob.glob("%s*.fasta"%inputs_folder)
    taxa = {}
    for item in input_list:
        records = list(SeqIO.parse(item, "fasta"))
        strain = os.path.splitext(os.path.basename(item))[0]
        if records:
            if "TPA_asm" in str(records[0]) or "MAG" in str(records[0]):
                genera = split_string_at(genera,":",1)[0]
            else:
                genera = split_string_at(records[0].description," ",1)[1]
            if "Candidatus" in str(genera):
                taxa[strain] = str(genera).partition(' ')[2].partition(' ')[0]
            else:
                taxa[strain] = str(genera).partition(' ')[0].lstrip("[").rstrip("]")
    return taxa
            
taxa_dict = extract_ncbi_taxa("./top_100/")

In [7]:
taxa_dict

{'NAPS01': 'Westiellopsis',
 'JAHHHK01': 'Westiellopsis',
 'NMQE01': 'Fischerella',
 'AJLJ01': 'Fischerella',
 'NQMD01': 'Fischerella',
 'JACYLX01': 'Fischerella',
 'MRCA01': 'Fischerella',
 'JADEXT01': 'Nostoc',
 'NHAE01': 'Fischerella',
 'WBKL01': 'Nostoc',
 'LAHF01': 'Nostoc',
 'JAHHHW01': 'Nostoc',
 'JACYLY01': 'Fischerella',
 'NGZZ01': 'Fischerella',
 'NHAD01': 'Fischerella',
 'LIRN01': 'Hapalosiphon',
 'ALVS01': 'Fischerella',
 'LAHG01': 'Nostoc',
 'AGIZ01': 'Fischerella',
 'NGZS01': 'Fischerella',
 'JACKZU01': 'Nostoc',
 'NQME01': 'Fischerella',
 'AJLK01': 'Fischerella',
 'NMQD01': 'Fischerella',
 'NHAF01': 'Fischerella',
 'MNPM02': 'Mastigocladus',
 'NGAA01': 'Fischerella',
 'NGZX01': 'Fischerella',
 'JAECZC01': 'Amazonocrinis',
 'LAHE01': 'Nostoc',
 'JACVRH01': 'Nostoc',
 'LXQE01': 'Nostoc',
 'ALVX01': 'Fischerella',
 'NMQF01': 'Fischerella',
 'NMQG01': 'Fischerella',
 'NQMF01': 'Fischerella',
 'JABXKH01': 'Nostoc',
 'JACJRZ01': 'Fischerella',
 'JADEXV01': 'Nostocales',
 'NHAG

In [16]:
fasta_glob = glob.glob("./top_100/*.fasta")

for fasta_path in fasta_glob:
    strain = os.path.basename(fasta_path.split('.fasta')[0])
    print('mv %s ./top_100/%s_%s.fasta'%(fasta_path,taxa_dict[strain],strain))
    subprocess.call('mv %s ./top_100/%s_%s.fasta'%(fasta_path,taxa_dict[strain],strain),shell=True)

mv ./top_100/NAPS01.fasta ./top_100/Westiellopsis_NAPS01.fasta
mv ./top_100/JAHHHK01.fasta ./top_100/Westiellopsis_JAHHHK01.fasta
mv ./top_100/NMQE01.fasta ./top_100/Fischerella_NMQE01.fasta
mv ./top_100/AJLJ01.fasta ./top_100/Fischerella_AJLJ01.fasta
mv ./top_100/NQMD01.fasta ./top_100/Fischerella_NQMD01.fasta
mv ./top_100/JACYLX01.fasta ./top_100/Fischerella_JACYLX01.fasta
mv ./top_100/MRCA01.fasta ./top_100/Fischerella_MRCA01.fasta
mv ./top_100/JADEXT01.fasta ./top_100/Nostoc_JADEXT01.fasta
mv ./top_100/NHAE01.fasta ./top_100/Fischerella_NHAE01.fasta
mv ./top_100/WBKL01.fasta ./top_100/Nostoc_WBKL01.fasta
mv ./top_100/LAHF01.fasta ./top_100/Nostoc_LAHF01.fasta
mv ./top_100/JAHHHW01.fasta ./top_100/Nostoc_JAHHHW01.fasta
mv ./top_100/JACYLY01.fasta ./top_100/Fischerella_JACYLY01.fasta
mv ./top_100/NGZZ01.fasta ./top_100/Fischerella_NGZZ01.fasta
mv ./top_100/NHAD01.fasta ./top_100/Fischerella_NHAD01.fasta
mv ./top_100/LIRN01.fasta ./top_100/Hapalosiphon_LIRN01.fasta
mv ./top_100/ALVS01

In [17]:
!mv /Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/Fischerella_CENA71.fasta ./top_100/

## Parsing input and generating reference (M. producens PAL)

Input: 21 housekeeping genes from complete genomes published here, [*Moorea* PNAS paper](http://www.pnas.org/content/114/12/3198.short)

In [18]:
!cp ./inputs/hk_genes_calteau_pnas/* ./pnas-phylogenomics-top_100/
!ls ./pnas-phylogenomics-top_100/*.faa | wc -l

      21


In [19]:
input_list = glob.glob("/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/*.faa")
output_handle = open("/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/PAL_core_genes.faa", "w")

count = 0
for item in natsorted(input_list):
    input_handle  = open(item,"r")
    new_id = os.path.basename(item).split("_scaffold")[0]
    for seq_record in SeqIO.parse(input_handle, "fasta"):
        if "Moorea_producens_PAL_15AUG08-1" in seq_record.id:
            count += 1
            new_id = str(os.path.basename(item).split(".faa")[0]) + str("_Moorea_producens_PAL15AUG081")
            output_handle.write(">%s %s\n%s\n" % (
               new_id,
               "",
               seq_record.seq))
    input_handle.close()

output_handle.close()

assert(len(input_list) == count)

In [21]:
!ls ./pnas-phylogenomics-top_100/*

./pnas-phylogenomics-top_100/10_rpsB.faa
./pnas-phylogenomics-top_100/11_rpsC.faa
./pnas-phylogenomics-top_100/12_rpsD.faa
./pnas-phylogenomics-top_100/13_rpsE.faa
./pnas-phylogenomics-top_100/14_rpsG.faa
./pnas-phylogenomics-top_100/15_rpsH.faa
./pnas-phylogenomics-top_100/16_rpsI.faa
./pnas-phylogenomics-top_100/17_rpsK.faa
./pnas-phylogenomics-top_100/18_rpsL.faa
./pnas-phylogenomics-top_100/19_rpsM.faa
./pnas-phylogenomics-top_100/1_rplA.faa
./pnas-phylogenomics-top_100/20_rpsO.faa
./pnas-phylogenomics-top_100/21_rpsQ.faa
./pnas-phylogenomics-top_100/2_rplC.faa
./pnas-phylogenomics-top_100/3_rplE.faa
./pnas-phylogenomics-top_100/4_rplF.faa
./pnas-phylogenomics-top_100/5_rplK.faa
./pnas-phylogenomics-top_100/6_rplM.faa
./pnas-phylogenomics-top_100/7_rplN.faa
./pnas-phylogenomics-top_100/8_rplR.faa
./pnas-phylogenomics-top_100/9_rplV.faa
./pnas-phylogenomics-top_100/PAL_core_genes.faa


## Annotating genomes with Prokka

In [22]:
!mkdir ./prokka

In [42]:
#conda install -c bioconda prokka

In [34]:
def run_prokka(input_fasta,prokka_folder):
    fasta_glob = glob.glob("%s*.fasta"%input_fasta)
    for fasta_path in fasta_glob:
        strain = os.path.basename(fasta_path).split('.')[0]
        output_folder = '%s%s/'%(prokka_folder,strain)
        prokka_cmd = "prokka %s --outdir %s --prefix %s --cpus 12"%(fasta_path,output_folder,strain)
        p = subprocess.Popen(prokka_cmd, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE,
                             stderr=subprocess.STDOUT, close_fds=True,cwd="./")
        output = p.stdout.read()
        print(output)
        subprocess.call('mv %s%s/%s.*.faa %s%s/%s.faa'%
                        (prokka_folder,strain,strain,prokka_folder,strain,strain),shell=True)
    
run_prokka("/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/top_100/",
           "/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/prokka/")

b"[10:02:10] This is prokka 1.14.6\n[10:02:10] Written by Torsten Seemann <torsten.seemann@gmail.com>\n[10:02:10] Homepage is https://github.com/tseemann/prokka\n[10:02:10] Local time is Mon Feb 21 10:02:10 2022\n[10:02:10] You are tiagoferreiraleao\n[10:02:10] Operating system is darwin\n[10:02:10] You have BioPerl 1.007002\n[10:02:10] System has 4 cores.\n[10:02:10] Option --cpu asked for 12 cores, but system only has 4\n[10:02:10] Will use maximum of 4 cores.\n[10:02:10] Annotating as >>> Bacteria <<<\n[10:02:10] Generating locus_tag from '/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/top_100/Fischerella_NMQG01.fasta' contents.\n[10:02:11] Setting --locustag KLMOHPHK from MD5 45681914da976b78ddc3663dbebd6d48\n[10:02:11] Creating new output folder: /Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/prokka/Fischerella_NMQG01/\n[10:02:11] Running: mkdir -p \\/Users\\/tiagoferreiraleao\\/Dropbox\\/tiago\\-NAS\\/schneider\\/prokka\\/Fischerella_NMQG01\\/\n[10:02:11] Using filena

b"[10:02:41] This is prokka 1.14.6\n[10:02:41] Written by Torsten Seemann <torsten.seemann@gmail.com>\n[10:02:41] Homepage is https://github.com/tseemann/prokka\n[10:02:41] Local time is Mon Feb 21 10:02:41 2022\n[10:02:41] You are tiagoferreiraleao\n[10:02:41] Operating system is darwin\n[10:02:41] You have BioPerl 1.007002\n[10:02:41] System has 4 cores.\n[10:02:41] Option --cpu asked for 12 cores, but system only has 4\n[10:02:41] Will use maximum of 4 cores.\n[10:02:41] Annotating as >>> Bacteria <<<\n[10:02:41] Generating locus_tag from '/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/top_100/Fischerella_NQMF01.fasta' contents.\n[10:02:41] Setting --locustag NOGBNIPK from MD5 780b729497921e37e556a305abdebbb3\n[10:02:41] Creating new output folder: /Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/prokka/Fischerella_NQMF01/\n[10:02:41] Running: mkdir -p \\/Users\\/tiagoferreiraleao\\/Dropbox\\/tiago\\-NAS\\/schneider\\/prokka\\/Fischerella_NQMF01\\/\n[10:02:41] Using filena

b"[10:03:07] This is prokka 1.14.6\n[10:03:07] Written by Torsten Seemann <torsten.seemann@gmail.com>\n[10:03:07] Homepage is https://github.com/tseemann/prokka\n[10:03:07] Local time is Mon Feb 21 10:03:07 2022\n[10:03:07] You are tiagoferreiraleao\n[10:03:07] Operating system is darwin\n[10:03:07] You have BioPerl 1.007002\n[10:03:08] System has 4 cores.\n[10:03:08] Option --cpu asked for 12 cores, but system only has 4\n[10:03:08] Will use maximum of 4 cores.\n[10:03:08] Annotating as >>> Bacteria <<<\n[10:03:08] Generating locus_tag from '/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/top_100/Nostocales_JADEXV01.fasta' contents.\n[10:03:08] Setting --locustag BMHCBODI from MD5 b61cb8d2e719a07ffcdca4ea4cdda202\n[10:03:08] Creating new output folder: /Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/prokka/Nostocales_JADEXV01/\n[10:03:08] Running: mkdir -p \\/Users\\/tiagoferreiraleao\\/Dropbox\\/tiago\\-NAS\\/schneider\\/prokka\\/Nostocales_JADEXV01\\/\n[10:03:08] Using fil

b"[10:03:36] This is prokka 1.14.6\n[10:03:36] Written by Torsten Seemann <torsten.seemann@gmail.com>\n[10:03:36] Homepage is https://github.com/tseemann/prokka\n[10:03:36] Local time is Mon Feb 21 10:03:36 2022\n[10:03:36] You are tiagoferreiraleao\n[10:03:36] Operating system is darwin\n[10:03:36] You have BioPerl 1.007002\n[10:03:36] System has 4 cores.\n[10:03:36] Option --cpu asked for 12 cores, but system only has 4\n[10:03:36] Will use maximum of 4 cores.\n[10:03:36] Annotating as >>> Bacteria <<<\n[10:03:36] Generating locus_tag from '/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/top_100/Fischerella_JACJRZ01.fasta' contents.\n[10:03:36] Setting --locustag COMBBOFO from MD5 c86bb8f8aa0885c0dc9d51118a17400a\n[10:03:36] Creating new output folder: /Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/prokka/Fischerella_JACJRZ01/\n[10:03:36] Running: mkdir -p \\/Users\\/tiagoferreiraleao\\/Dropbox\\/tiago\\-NAS\\/schneider\\/prokka\\/Fischerella_JACJRZ01\\/\n[10:03:36] Using 

b"[10:04:05] This is prokka 1.14.6\n[10:04:05] Written by Torsten Seemann <torsten.seemann@gmail.com>\n[10:04:05] Homepage is https://github.com/tseemann/prokka\n[10:04:05] Local time is Mon Feb 21 10:04:05 2022\n[10:04:05] You are tiagoferreiraleao\n[10:04:05] Operating system is darwin\n[10:04:05] You have BioPerl 1.007002\n[10:04:06] System has 4 cores.\n[10:04:06] Option --cpu asked for 12 cores, but system only has 4\n[10:04:06] Will use maximum of 4 cores.\n[10:04:06] Annotating as >>> Bacteria <<<\n[10:04:06] Generating locus_tag from '/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/top_100/Nostoc_JACVRH01.fasta' contents.\n[10:04:06] Setting --locustag PLFJKCIB from MD5 95f34c2bc6f5610527e051db8da17888\n[10:04:06] Creating new output folder: /Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/prokka/Nostoc_JACVRH01/\n[10:04:06] Running: mkdir -p \\/Users\\/tiagoferreiraleao\\/Dropbox\\/tiago\\-NAS\\/schneider\\/prokka\\/Nostoc_JACVRH01\\/\n[10:04:06] Using filename prefix

...

b"[10:56:23] This is prokka 1.14.6\n[10:56:23] Written by Torsten Seemann <torsten.seemann@gmail.com>\n[10:56:23] Homepage is https://github.com/tseemann/prokka\n[10:56:23] Local time is Mon Feb 21 10:56:23 2022\n[10:56:23] You are tiagoferreiraleao\n[10:56:23] Operating system is darwin\n[10:56:23] You have BioPerl 1.007002\n[10:56:23] System has 4 cores.\n[10:56:23] Option --cpu asked for 12 cores, but system only has 4\n[10:56:23] Will use maximum of 4 cores.\n[10:56:23] Annotating as >>> Bacteria <<<\n[10:56:23] Generating locus_tag from '/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/top_100/Fischerella_NHAK01.fasta' contents.\n[10:56:23] Setting --locustag LHKAMFKC from MD5 514a6f4c51372f49561ae50dea9cedac\n[10:56:23] Creating new output folder: /Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/prokka/Fischerella_NHAK01/\n[10:56:23] Running: mkdir -p \\/Users\\/tiagoferreiraleao\\/Dropbox\\/tiago\\-NAS\\/schneider\\/prokka\\/Fischerella_NHAK01\\/\n[10:56:23] Using filena

b"[10:56:54] This is prokka 1.14.6\n[10:56:54] Written by Torsten Seemann <torsten.seemann@gmail.com>\n[10:56:54] Homepage is https://github.com/tseemann/prokka\n[10:56:54] Local time is Mon Feb 21 10:56:54 2022\n[10:56:54] You are tiagoferreiraleao\n[10:56:54] Operating system is darwin\n[10:56:54] You have BioPerl 1.007002\n[10:56:55] System has 4 cores.\n[10:56:55] Option --cpu asked for 12 cores, but system only has 4\n[10:56:55] Will use maximum of 4 cores.\n[10:56:55] Annotating as >>> Bacteria <<<\n[10:56:55] Generating locus_tag from '/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/top_100/Fischerella_NMQB01.fasta' contents.\n[10:56:55] Setting --locustag JGICIIKI from MD5 302c2242ac780daa824fbf206ad0c845\n[10:56:55] Creating new output folder: /Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/prokka/Fischerella_NMQB01/\n[10:56:55] Running: mkdir -p \\/Users\\/tiagoferreiraleao\\/Dropbox\\/tiago\\-NAS\\/schneider\\/prokka\\/Fischerella_NMQB01\\/\n[10:56:55] Using filena

b"[10:57:16] This is prokka 1.14.6\n[10:57:16] Written by Torsten Seemann <torsten.seemann@gmail.com>\n[10:57:16] Homepage is https://github.com/tseemann/prokka\n[10:57:16] Local time is Mon Feb 21 10:57:16 2022\n[10:57:16] You are tiagoferreiraleao\n[10:57:16] Operating system is darwin\n[10:57:16] You have BioPerl 1.007002\n[10:57:16] System has 4 cores.\n[10:57:16] Option --cpu asked for 12 cores, but system only has 4\n[10:57:16] Will use maximum of 4 cores.\n[10:57:16] Annotating as >>> Bacteria <<<\n[10:57:16] Generating locus_tag from '/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/top_100/Mastigocladus_MNPM02.fasta' contents.\n[10:57:16] Setting --locustag PGJICEHD from MD5 9032ce1d991eb6ed691aad33cbfe3e25\n[10:57:16] Creating new output folder: /Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/prokka/Mastigocladus_MNPM02/\n[10:57:16] Running: mkdir -p \\/Users\\/tiagoferreiraleao\\/Dropbox\\/tiago\\-NAS\\/schneider\\/prokka\\/Mastigocladus_MNPM02\\/\n[10:57:16] Using 

b"[10:57:40] This is prokka 1.14.6\n[10:57:40] Written by Torsten Seemann <torsten.seemann@gmail.com>\n[10:57:40] Homepage is https://github.com/tseemann/prokka\n[10:57:40] Local time is Mon Feb 21 10:57:40 2022\n[10:57:40] You are tiagoferreiraleao\n[10:57:40] Operating system is darwin\n[10:57:40] You have BioPerl 1.007002\n[10:57:40] System has 4 cores.\n[10:57:40] Option --cpu asked for 12 cores, but system only has 4\n[10:57:40] Will use maximum of 4 cores.\n[10:57:40] Annotating as >>> Bacteria <<<\n[10:57:40] Generating locus_tag from '/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/top_100/Nostoc_LAHH01.fasta' contents.\n[10:57:40] Setting --locustag HIMFHEKA from MD5 126f1e4a4415e2f3e4ff00d6eb1b335d\n[10:57:40] Creating new output folder: /Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/prokka/Nostoc_LAHH01/\n[10:57:40] Running: mkdir -p \\/Users\\/tiagoferreiraleao\\/Dropbox\\/tiago\\-NAS\\/schneider\\/prokka\\/Nostoc_LAHH01\\/\n[10:57:40] Using filename prefix: Nost

b"[10:58:28] This is prokka 1.14.6\n[10:58:28] Written by Torsten Seemann <torsten.seemann@gmail.com>\n[10:58:28] Homepage is https://github.com/tseemann/prokka\n[10:58:28] Local time is Mon Feb 21 10:58:28 2022\n[10:58:28] You are tiagoferreiraleao\n[10:58:28] Operating system is darwin\n[10:58:28] You have BioPerl 1.007002\n[10:58:28] System has 4 cores.\n[10:58:28] Option --cpu asked for 12 cores, but system only has 4\n[10:58:28] Will use maximum of 4 cores.\n[10:58:28] Annotating as >>> Bacteria <<<\n[10:58:28] Generating locus_tag from '/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/top_100/Nostoc_JABXKL01.fasta' contents.\n[10:58:28] Setting --locustag GPGBLMAP from MD5 090b56a9325beed67f4908813cc36b74\n[10:58:28] Creating new output folder: /Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/prokka/Nostoc_JABXKL01/\n[10:58:28] Running: mkdir -p \\/Users\\/tiagoferreiraleao\\/Dropbox\\/tiago\\-NAS\\/schneider\\/prokka\\/Nostoc_JABXKL01\\/\n[10:58:28] Using filename prefix

## Identify housekeeping genes in prokka annotation for final scaffolds

In [11]:
#conda install -c bioconda diamond

In [35]:
!mkdir ./pnas-phylogenomics-top_100/diamond

In [36]:
!diamond makedb --in ./pnas-phylogenomics-top_100/PAL_core_genes.faa -d ./pnas-phylogenomics-top_100/diamond/PAL_hkg

diamond v0.9.24.125 | by Benjamin Buchfink <buchfink@gmail.com>
Licensed under the GNU GPL <https://www.gnu.org/licenses/gpl.txt>
Check http://github.com/bbuchfink/diamond for updates.

#CPU threads: 8
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database file: ./pnas-phylogenomics-top_100/PAL_core_genes.faa
Opening the database file...  [0.001633s]
Loading sequences...  [0.00035s]
Masking sequences...  [0.000751s]
Writing sequences...  [0.000106s]
Hashing sequences...  [8e-06s]
Loading sequences...  [9e-06s]
Writing trailer...  [4e-06s]
Closing the input file...  [2.1e-05s]
Closing the database file...  [0.00102s]
Database hash = cf7016f4bf64255b40ca9dd62558c2d2
Processed 21 sequences, 3334 letters.
Total time = 0.003997s


In [37]:
def run_diamond(database,input_fasta,output_path):
    output_file = os.path.join(output_path, os.path.splitext(os.path.basename(input_fasta))[0] + ".matches")
    diamond_cmd = "diamond blastp -d %s -q %s -a %s"%(database,input_fasta,output_file)
    p = subprocess.Popen(diamond_cmd, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE,
                         stderr=subprocess.STDOUT, close_fds=True,cwd="./")
    output = p.stdout.read()
    print(output)

def view_diamond(diamond_matches,m8_output):
    view_cmd = "diamond view -a %s -o %s"%(diamond_matches,m8_output)
    p = subprocess.Popen(view_cmd, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE,
                         stderr=subprocess.STDOUT, close_fds=True,cwd="./")
    output = p.stdout.read()
    print(output)
    
outdir = "/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/diamond/"
dbfile = "/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/diamond/PAL_hkg"
prokka_list = glob.glob("/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/prokka/*/*.faa")
scaffold_list = glob.glob("/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/top_100/*.fasta")

strain_list = []
for item in scaffold_list:
    strain = os.path.basename(item).split(".")[0]
    strain_list.append(strain)
    
count = 0
    
for prokka_faa in natsorted(prokka_list):
    strain = os.path.basename(prokka_faa).split(".")[0]
    if strain in strain_list:
        count += 1
        run_diamond(dbfile,
                    prokka_faa,
                    outdir)
        view_diamond("%s%s.matches.daa"%(outdir,strain),"%s%s.m8"%(outdir,strain))

count

b'diamond v0.9.24.125 | by Benjamin Buchfink <buchfink@gmail.com>\nLicensed under the GNU GPL <https://www.gnu.org/licenses/gpl.txt>\nCheck http://github.com/bbuchfink/diamond for updates.\n\n#CPU threads: 8\nScoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)\nTemporary directory: /Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/diamond\nOpening the database...  [0.000416s]\n#Target sequences to report alignments for: 25\nOpening the input file...  [0.000312s]\nOpening the output file...  [6.1e-05s]\nLoading query sequences...  [0.013829s]\nMasking queries...  [0.089515s]\nBuilding query seed set...  [0.018152s]\nAlgorithm: Double-indexed\nBuilding query histograms...  [0.014318s]\nAllocating buffers...  [2.3e-05s]\nLoading reference sequences...  [0.000439s]\nBuilding reference histograms...  [0.000306s]\nAllocating buffers...  [3.2e-05s]\nInitializing temporary storage...  [0.044864s]\nProcessing query chunk 0, reference chunk 0, 

b'diamond v0.9.24.125 | by Benjamin Buchfink <buchfink@gmail.com>\nLicensed under the GNU GPL <https://www.gnu.org/licenses/gpl.txt>\nCheck http://github.com/bbuchfink/diamond for updates.\n\n#CPU threads: 8\nScoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)\nTemporary directory: /Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/diamond\nOpening the database...  [8e-05s]\n#Target sequences to report alignments for: 25\nOpening the input file...  [0.000455s]\nOpening the output file...  [6.1e-05s]\nLoading query sequences...  [0.015417s]\nMasking queries...  [0.067726s]\nBuilding query seed set...  [0.017135s]\nAlgorithm: Double-indexed\nBuilding query histograms...  [0.018127s]\nAllocating buffers...  [3.7e-05s]\nLoading reference sequences...  [5.1e-05s]\nBuilding reference histograms...  [0.00027s]\nAllocating buffers...  [2e-05s]\nInitializing temporary storage...  [0.016862s]\nProcessing query chunk 0, reference chunk 0, shape 0

b'diamond v0.9.24.125 | by Benjamin Buchfink <buchfink@gmail.com>\nLicensed under the GNU GPL <https://www.gnu.org/licenses/gpl.txt>\nCheck http://github.com/bbuchfink/diamond for updates.\n\n#CPU threads: 8\nScoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)\nTemporary directory: /Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/diamond\nOpening the database...  [0.000177s]\n#Target sequences to report alignments for: 25\nOpening the input file...  [0.00045s]\nOpening the output file...  [0.000115s]\nLoading query sequences...  [0.018785s]\nMasking queries...  [0.102642s]\nBuilding query seed set...  [0.024189s]\nAlgorithm: Double-indexed\nBuilding query histograms...  [0.013448s]\nAllocating buffers...  [2.2e-05s]\nLoading reference sequences...  [5.7e-05s]\nBuilding reference histograms...  [0.000187s]\nAllocating buffers...  [1.5e-05s]\nInitializing temporary storage...  [0.024586s]\nProcessing query chunk 0, reference chunk 0, s

b'diamond v0.9.24.125 | by Benjamin Buchfink <buchfink@gmail.com>\nLicensed under the GNU GPL <https://www.gnu.org/licenses/gpl.txt>\nCheck http://github.com/bbuchfink/diamond for updates.\n\n#CPU threads: 8\nScoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)\nTemporary directory: /Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/diamond\nOpening the database...  [0.000184s]\n#Target sequences to report alignments for: 25\nOpening the input file...  [0.000783s]\nOpening the output file...  [0.000108s]\nLoading query sequences...  [0.018481s]\nMasking queries...  [0.0839s]\nBuilding query seed set...  [0.024214s]\nAlgorithm: Double-indexed\nBuilding query histograms...  [0.014593s]\nAllocating buffers...  [4e-05s]\nLoading reference sequences...  [4.1e-05s]\nBuilding reference histograms...  [0.000187s]\nAllocating buffers...  [1.5e-05s]\nInitializing temporary storage...  [0.029871s]\nProcessing query chunk 0, reference chunk 0, shap

b'diamond v0.9.24.125 | by Benjamin Buchfink <buchfink@gmail.com>\nLicensed under the GNU GPL <https://www.gnu.org/licenses/gpl.txt>\nCheck http://github.com/bbuchfink/diamond for updates.\n\n#CPU threads: 8\nScoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)\nTemporary directory: /Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/diamond\nOpening the database...  [0.00012s]\n#Target sequences to report alignments for: 25\nOpening the input file...  [0.000425s]\nOpening the output file...  [9.5e-05s]\nLoading query sequences...  [0.011998s]\nMasking queries...  [0.053226s]\nBuilding query seed set...  [0.015283s]\nAlgorithm: Double-indexed\nBuilding query histograms...  [0.010683s]\nAllocating buffers...  [3.5e-05s]\nLoading reference sequences...  [5.5e-05s]\nBuilding reference histograms...  [0.000268s]\nAllocating buffers...  [2.6e-05s]\nInitializing temporary storage...  [0.029417s]\nProcessing query chunk 0, reference chunk 0, sh

...

b'diamond v0.9.24.125 | by Benjamin Buchfink <buchfink@gmail.com>\nLicensed under the GNU GPL <https://www.gnu.org/licenses/gpl.txt>\nCheck http://github.com/bbuchfink/diamond for updates.\n\n#CPU threads: 8\nScoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)\nTemporary directory: /Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/diamond\nOpening the database...  [0.000158s]\n#Target sequences to report alignments for: 25\nOpening the input file...  [0.000212s]\nOpening the output file...  [0.00012s]\nLoading query sequences...  [0.016429s]\nMasking queries...  [0.09546s]\nBuilding query seed set...  [0.028549s]\nAlgorithm: Double-indexed\nBuilding query histograms...  [0.016503s]\nAllocating buffers...  [3.3e-05s]\nLoading reference sequences...  [4.2e-05s]\nBuilding reference histograms...  [0.000219s]\nAllocating buffers...  [2.6e-05s]\nInitializing temporary storage...  [0.03164s]\nProcessing query chunk 0, reference chunk 0, sha

b'diamond v0.9.24.125 | by Benjamin Buchfink <buchfink@gmail.com>\nLicensed under the GNU GPL <https://www.gnu.org/licenses/gpl.txt>\nCheck http://github.com/bbuchfink/diamond for updates.\n\n#CPU threads: 8\nScoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)\nTemporary directory: /Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/diamond\nOpening the database...  [0.00014s]\n#Target sequences to report alignments for: 25\nOpening the input file...  [0.000626s]\nOpening the output file...  [8.2e-05s]\nLoading query sequences...  [0.018761s]\nMasking queries...  [0.109021s]\nBuilding query seed set...  [0.0237s]\nAlgorithm: Double-indexed\nBuilding query histograms...  [0.020402s]\nAllocating buffers...  [3.5e-05s]\nLoading reference sequences...  [5.5e-05s]\nBuilding reference histograms...  [0.000294s]\nAllocating buffers...  [2.6e-05s]\nInitializing temporary storage...  [0.03051s]\nProcessing query chunk 0, reference chunk 0, shape

b'diamond v0.9.24.125 | by Benjamin Buchfink <buchfink@gmail.com>\nLicensed under the GNU GPL <https://www.gnu.org/licenses/gpl.txt>\nCheck http://github.com/bbuchfink/diamond for updates.\n\n#CPU threads: 8\nScoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)\nTemporary directory: /Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/diamond\nOpening the database...  [0.000159s]\n#Target sequences to report alignments for: 25\nOpening the input file...  [0.000238s]\nOpening the output file...  [0.000108s]\nLoading query sequences...  [0.014491s]\nMasking queries...  [0.093875s]\nBuilding query seed set...  [0.018512s]\nAlgorithm: Double-indexed\nBuilding query histograms...  [0.013808s]\nAllocating buffers...  [3.5e-05s]\nLoading reference sequences...  [5.6e-05s]\nBuilding reference histograms...  [0.000249s]\nAllocating buffers...  [2.5e-05s]\nInitializing temporary storage...  [0.024876s]\nProcessing query chunk 0, reference chunk 0, 

b'diamond v0.9.24.125 | by Benjamin Buchfink <buchfink@gmail.com>\nLicensed under the GNU GPL <https://www.gnu.org/licenses/gpl.txt>\nCheck http://github.com/bbuchfink/diamond for updates.\n\n#CPU threads: 8\nScoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)\nTemporary directory: /Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/diamond\nOpening the database...  [0.000151s]\n#Target sequences to report alignments for: 25\nOpening the input file...  [0.000212s]\nOpening the output file...  [8.1e-05s]\nLoading query sequences...  [0.020646s]\nMasking queries...  [0.091404s]\nBuilding query seed set...  [0.025436s]\nAlgorithm: Double-indexed\nBuilding query histograms...  [0.025459s]\nAllocating buffers...  [3.8e-05s]\nLoading reference sequences...  [6.3e-05s]\nBuilding reference histograms...  [0.000364s]\nAllocating buffers...  [4.9e-05s]\nInitializing temporary storage...  [0.022351s]\nProcessing query chunk 0, reference chunk 0, s

105

## Selecting genes (the highest %ID homolog per PAL housekeeping gene) and adding to existent aligments

In [38]:
diamond_columns = ["qseqid","sseqid","pident","length","mismatch","gapopen",
                   "qstart","qend","sstart","send","evalue","bitscore"]
                    #retrived from https://www.biostars.org/p/166013/

In [39]:
def get_hkg_dict(m8_file):
    prokka_hkg = {}
    diamond_df = pd.read_csv(m8_file,sep="\t",names=diamond_columns).sort_values(by="sseqid")
    for i,r in diamond_df.iterrows():
        if diamond_df.qseqid.loc[i] not in list(prokka_hkg.values()):
            subset = diamond_df[diamond_df.sseqid == diamond_df.sseqid.loc[i]]
            max_id = subset.loc[subset['pident'].idxmax()]["pident"]
            max_qseq = subset.loc[subset['pident'].idxmax()]["qseqid"]
            max_sseq = subset.loc[subset['pident'].idxmax()]["sseqid"]
            prokka_hkg[max_qseq] = max_sseq
    assert(len(prokka_hkg.keys()) == len(np.unique(diamond_df.sseqid.values)))
    return prokka_hkg

def extract_genes(m8_file,prokka_folder,hkg_folder,prokka_hkg):
    strain = os.path.basename(m8_file).split(".")[0]
    prokka_faa = "%s%s/%s.faa"%(prokka_folder,strain,strain)
    input_handle  = open(prokka_faa,"r")
    for seq_record in SeqIO.parse(input_handle, "fasta"):
        if int(seq_record.id) in prokka_hkg:
            hkg_homolog = str(prokka_hkg[int(seq_record.id)].split("_Moorea")[0])
            output_handle = open("%s%s.faa"%(hkg_folder,hkg_homolog), "a")
            new_id = "%s"%(strain)
            output_handle.write("\n\n>%s %s\n%s\n" % (
               new_id,
               "",
               seq_record.seq))
            output_handle.close()
    input_handle.close()

m8_list = glob.glob("/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/diamond/*.m8")
prokka_folder = "/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/prokka/"
hkg_folder = "/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/"

complete_count = 0
dict_len = []

for m8_file in m8_list:
    prokka_hkg = get_hkg_dict(m8_file)
    dict_len.append(len(prokka_hkg))
    if len(prokka_hkg) == 21:
        complete_count += 1
        extract_genes(m8_file,prokka_folder,hkg_folder,prokka_hkg)

print(complete_count)
print(np.average(dict_len))

93
20.79


In [40]:
len(m8_list)

100

**Running MUSCLE and TrimAl**

In [ ]:
#conda install -c bioconda muscle
#conda install -c bioconda trimal

In [42]:
input_list

['/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/11_rpsC.faa',
 '/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/10_rpsB.faa',
 '/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/4_rplF.faa',
 '/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/15_rpsH.faa',
 '/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/20_rpsO.faa',
 '/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/18_rpsL.faa',
 '/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/19_rpsM.faa',
 '/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/9_rplV.faa',
 '/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/2_rplC.faa',
 '/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/6_rplM.faa',
 '/Users/tiagoferreiraleao/Dropbox/tiago-NAS

In [43]:
for fasta_file in input_list:
    muscle_in = fasta_file
    muscle_out = os.path.join(os.path.dirname(fasta_file),os.path.basename(fasta_file).strip(".faa")+".ali.faa")
    trim_out = os.path.join(os.path.dirname(fasta_file),os.path.basename(fasta_file).strip(".faa")+".trimmed.faa")
    subprocess.call("muscle -in %s -out %s"%(muscle_in,muscle_out),shell=True)
    subprocess.call("trimal -in %s -out %s -gt 0.8 -st 0.001 -cons 70"%(muscle_out,trim_out),shell=True)

In [45]:
!ls ./pnas-phylogenomics-top_100/

10_rpsB.ali.faa     15_rpsH.trimmed.faa 20_rpsO.faa         6_rplM.ali.faa
10_rpsB.faa         16_rpsI.ali.faa     20_rpsO.trimmed.faa 6_rplM.faa
10_rpsB.trimmed.faa 16_rpsI.faa         21_rpsQ.ali.faa     6_rplM.trimmed.faa
11_rpsC.ali.faa     16_rpsI.trimmed.faa 21_rpsQ.faa         7_rplN.ali.faa
11_rpsC.faa         17_rpsK.ali.faa     21_rpsQ.trimmed.faa 7_rplN.faa
11_rpsC.trimmed.faa 17_rpsK.faa         2_rplC.ali.faa      7_rplN.trimmed.faa
12_rpsD.ali.faa     17_rpsK.trimmed.faa 2_rplC.faa          8_rplR.ali.faa
12_rpsD.faa         18_rpsL.ali.faa     2_rplC.trimmed.faa  8_rplR.faa
12_rpsD.trimmed.faa 18_rpsL.faa         3_rplE.ali.faa      8_rplR.trimmed.faa
13_rpsE.ali.faa     18_rpsL.trimmed.faa 3_rplE.faa          9_rplV.ali.faa
13_rpsE.faa         19_rpsM.ali.faa     3_rplE.trimmed.faa  9_rplV.faa
13_rpsE.trimmed.faa 19_rpsM.faa         4_rplF.ali.faa      9_rplV.trimmed.faa
14_rpsG.ali.faa     19_rpsM.trimmed.faa 4_rplF.faa          PAL_core_genes.faa
14_rpsG.faa         1

**Concatenating processed aligments**

In [46]:
ali_list = natsorted(glob.glob("/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/*.trimmed.faa"))

sequences_dict = {}

for item in ali_list:
    print(item)
    for record in (SeqIO.parse(item, "fasta")):
        if record.id not in sequences_dict:
            sequences_dict[record.id] = '%s'%record.seq
        else:
            sequences_dict[record.id] = '%s'%sequences_dict[record.id]+'%s'%record.seq
            
sequences_dict

/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/1_rplA.trimmed.faa
/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/2_rplC.trimmed.faa
/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/3_rplE.trimmed.faa
/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/4_rplF.trimmed.faa
/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/5_rplK.trimmed.faa
/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/6_rplM.trimmed.faa
/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/7_rplN.trimmed.faa
/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/8_rplR.trimmed.faa
/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/9_rplV.trimmed.faa
/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/10_rpsB.trimmed.faa
/Users/ti

{'Obscuribacter_phosphatis_(Mle1_12)': 'TKRRKKLDFEH-NQVAEEAIALKKENSVKFQTVEVHFKLGVNPKLNDQQVRSTVNLPGGTGKVRVAVVAKGDKLEASGADHFGAEELVEKIGAGLDFDLVATPDAMAMLSKMGKILGPKGLMPNPKDGTVTDVGTVELKAGKVSFRAEKDGIVQMIGKLSFDARLLNLAAVVDQVQKVKPSSVKGTYIKSYVSTTMGPGVKLDLNTLQDLSMTLGVMGKKVGMTQVFDENGLAVPVTVVKLGDNIVTEVKTKDKHGYQAVQVGGFKVAEKKLNKPELGGFKKKELAPMQPLKEFRLKDACAYNVGEALKEEILSAGMRVDVRGRSIGKGFQGTIKRYNAGRGPMSHGSKFHRSMGSIGAGTTPGRVFRGLHMPGHMGDDNTCARRIKVVKVDTEKGLLLVKGSIPGCDGALVVVTPSKKWN-----DIKYKQVAHLKQFYQNEMQVPRVVKVVINMGLGEAANNAKAIDSGVKELAIAGQKPVVNRARKSIATFKLRKGQPVGASVTLRGKRMYDFLAKLIHVALPRIRDFRGLSSKSFDGRGNFSVGIKEQLIFPEINYEQIDAVRGMDISIVTTAPTDQEAHALLAAMGMPFK-MSRIGKAPITVPAGVTVQFADNKVSVKGPKGQLTKEFRPEIAFKQEGDTITVLRKSEDRSVR--SLHGLSRTLLANMVKGVSTGFDKTLEIVGVGYRASMDGKKLVMQLGYSHSVEIDAPEGITIAVGKGNVITVSGADKQAVGDMSSFIRGKRPPEVYKGKGVKYQGEYIRRKAGKAGKKK---VKKVVGKVKLQIQAGKANPAPPIGPALGQHGVNIMQFCKEYNAKTQDKAGSVVPVEITVYEDRSFDFVLKTPPAAELIKKAANVEKGSAAPNKNKVGTLSRAKVTEIAKVKMPDLNATTLEAAESMIQGTARNMGITIVD-KTCPEVTRWLVVDAGKTLGRLAAEVALLRGKHKPFTPH

In [47]:
seq = []

for key in sequences_dict:
    seq.append(SeqRecord(Seq("%s"%sequences_dict[key],IUPAC.protein), id = "%s"%key, description=""))

for record in seq:
    print(len(record))
    
handle = open("/Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/concat_genes-220221.faa", "w")
SeqIO.write(seq,handle,"fasta")
handle.close

3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487
3487


<function TextIOWrapper.close()>

In [48]:
!grep '>' /Users/tiagoferreiraleao/Dropbox/tiago-NAS/schneider/pnas-phylogenomics-top_100/concat_genes-220221.faa

>Obscuribacter_phosphatis_(Mle1_12)
>Cyanobacterium_sp._(Zagget_111_MP)
>YS2_(Zagget_bin_221)
>SM1_D11_(SM2F09_MP_sspace)
>Gloeobacter_kilaueensis_JS1
>Gloeobacter_violaceus_PCC_7421_(re-annotation)
>Synechococcus_sp._RCC307_(re-annotation)
>Prochlorococcus_marinus_marinus_CCMP1375_(re-annotation)
>Cyanobium_gracile_PCC_6307
>Prochlorococcus_sp._MIT9211_(re-annotation)
>Prochlorococcus_marinus_str._MIT_9303
>Prochlorococcus_marinus_str._MIT_9313
>Synechococcus_sp._WH7803_(re-annotation)
>Synechococcus_sp._WH_8020
>Synechococcus_sp._CC9311
>Synechococcus_sp._WH_8016
>Synechococcus_sp._CC9902
>Synechococcus_sp._CC9616
>Synechococcus_sp._WH_8109
>Synechococcus_sp._CC9605
>Synechococcus_sp._KORDI-52_(genome_sequencing)
>Synechococcus_sp._WH_8102
>Synechococcus_sp._KORDI-100_(genome_sequencing)
>Synechococcus_sp._KORDI-49_(genome_sequencing)
>Prochlorococcus_sp._MIT0801
>Prochlorococcus_marinus_str._NATL2A
>Prochlorococcus_marinus_str._NATL1A
>Prochlorococcus_marinus_str._MIT_9215
>Prochlor